In [588]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

In [589]:
# model=Sequential()
# model.add(BatchNormalization())
# model.add(Dropout(0.20))
# model.add(Dense(2048,activation='relu',input_dim=train.shape[1],kernel_initializer='uniform'))
# model.add(BatchNormalization())
# model.add(Dense(2048,activation='tanh'))
# model.add(Dropout(0.2))
# model.add(Dense(1048,activation='tanh'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# #model.add(Dense(1024,kernel_initializer='uniform',activation='tanh'))
# #model.add(Dropout(0.5))
# model.add(Dense(target.shape[1],activation='sigmoid'))


# sgd=SGD(lr=0.01,momentum=0.9)
# model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
# model.fit(train.values,target.values,batch_size=60,epochs=100,verbose=2)



In [590]:
df = pd.read_csv('data/serieA_matches.csv')
df_all_matches = pd.read_csv('data/all_matches.csv')

In [591]:
home_teams = df_all_matches[['Home', 'Venue']].drop_duplicates().values
team_stadium = {}
for data in home_teams:
     team_stadium[data[0]] = data[1]

In [592]:
df['is_home'] = team_stadium[df['team'].iloc[0]]
df['is_home'] = df['stadium'] == df['is_home']
df['is_home'] = df['is_home'].astype(int)

In [593]:
cols_types = df.dtypes
cols_types[cols_types == object]

team       object
result     object
stadium    object
state      object
UF         object
date       object
time       object
dtype: object

In [594]:
#

In [595]:
df.head(2)

,fouls,corners,crosses,touches,tackles,interceptions,aerials_won,clearances,offsides,goal_kicks,...,fouls_commited_mean_attack,fouls_drawn_mean_attack,offsides_mean_attack,crosses_mean_attack,tackles_on_mean_attack,interceptions_mean_attack,own_goals_mean_attack,penalty_kicks_won_mean_attack,penalty_kicks_conceded_mean_attack,is_home
0,12.0,3.0,16.0,455,7.0,9.0,8.0,6.0,1.0,6.0,...,1.0,0.0,0.00,0.5,0.00,0.00,0.0,NaN,NaN,1
1,7.0,4.0,8.0,590,8.0,5.0,9.0,13.0,1.0,8.0,...,0.0,1.5,0.25,0.5,0.75,0.25,0.0,NaN,NaN,1


In [596]:
# df['date'] = pd.to_datetime(df['date'])
df['time'] = df['time'].apply(lambda time: time.split(':')[0]).astype(int)

In [597]:
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month
# df['quarter'] = df['date'].dt.quarter
# df['day'] = df['date'].dt.day

In [598]:
cols_to_remove = [
    'result',
    "date",
#     'team',
    "stadium",
    "state",
    "UF",
#     "match_id"
]
y_col = 'score'

In [599]:
df.drop(columns=cols_to_remove, inplace=True)

In [600]:
team_dict = {}
for idx, team in enumerate(df['team'].unique()):
    team_dict[team] = idx

In [601]:
feature_cols = [col for col in df.columns if col not in cols_to_remove]

In [602]:
df_teams = pd.DataFrame()
for team in df['team'].unique():
    cols_to_drop = ['is_home', 'team', 'score']

    df_team = df[df['team'] == team].copy().reset_index(drop=True)
    df_others_teams = df[
        (df['team'] != team) &
        (df['match_id'].isin(df_team['match_id'].tolist()))
    ].copy().reset_index(drop=True)
    
    columns = [col for col in df_team if col not in cols_to_drop]
    df_team.loc[df_team.index >= 3, columns] = (
        df_team[columns]
        .shift(1)
        .rolling(3)
        .mean()
        .iloc[3:]
    )

    df_others_teams.loc[df_others_teams.index >= 3, columns] = (
        df_team[columns]
        .shift(1)
        .rolling(3)
        .mean()
        .iloc[3:]
    )

    df_others_teams.drop(columns=cols_to_drop, inplace=True)
    df_others_teams.columns = [
        col + '_adversary' if col != 'match_id' else col for col in df_others_teams.columns
    ]
    
    df_teams = df_teams.append(
        df_team.merge(df_others_teams, on='match_id')
    )
df_teams['team'] = df_teams['team'].apply(lambda team: team_dict[team])

In [603]:
df_teams['score'].fillna(-1, inplace=True)
df_teams = df_teams[df_teams['score'] != -1]
df_teams['score'] = df_teams['score'].astype(int)

In [604]:
X = df_teams.drop(columns=['score'])
Y = df_teams['score']

In [605]:
# score
# result

# df.drop(columns=['result'], inplace=True)

In [606]:
# df.drop(columns=cols_to_remove, inplace=True)

In [607]:
# df

In [608]:
# model = XGBClassifier()

# rfe = RFE(model, 25)
# fit = rfe.fit(X, Y)
# print("Num Features: %d" % fit.n_features_)
# print("Selected Features: %s" % fit.support_)
# print("Feature Ranking: %s" % fit.ranking_)

In [609]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif

# test = SelectKBest(score_func=f_classif, k=25)
# fit = test.fit(X, Y)
# # summarize scores
# print(fit.scores_)
# features = fit.transform(X)

In [610]:
# cols = np.array(np.array(X.columns.tolist()))[list(fit.support_)]

In [611]:
# X = X[cols]

In [612]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

In [613]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    difference = list(np.abs((y_true - y_pred) / y_true))
    difference = [diff if diff != np.inf and not pd.isnull(diff) else 0 for diff in difference]
    return np.mean(difference)


In [614]:
def bias_percentage(y_true, y_pred): 
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    return  np.sum(y_pred) / np.sum(y_true)

In [615]:
X.sort_values(['matchweek', 'team'], inplace=True)

In [616]:
# n_cols_list = [5, 15, 25, 35, 45, 50, 63, 72, 84, 90, 100]
# for n_cols in n_cols_list:
n_cols = 50

model = XGBRegressor()

rfe = RFE(model, n_cols)
fit = rfe.fit(X, Y)

cols = np.array(np.array(X.columns.tolist()))[list(fit.support_)]
X_new = X[cols]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:67: FutureWarning: Pass n_features_to_select=50 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [617]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.1, random_state=7)

In [ ]:
%%time
parameters = {
    'eta': [0.05, 0.1, 0.3],
    'max_depth': [5, 6, 9, 12, 20],
    'subsample': [0.7, 0.6, 0.8, 1.0],
    'colsample_bytree': [0.3, 0.6, 0.8, 1.0],
    'min_child_weight': [1, 5, 8, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'n_estimators': [50, 100, 150, 500, 1000]
}

clf = GridSearchCV(model, parameters, n_jobs=1, cv=3)

clf.fit(X_train, y_train)

best_parameters = clf.best_params_

model = XGBRegressor(**best_parameters)
# model = XGBRegressor()

In [ ]:
from sklearn.metrics import mean_absolute_error


model.fit(X_train, y_train)

y_pred = list(model.predict(X_test))
predictions = list([round(value) for value in y_pred])

# evaluate predictions
mape = mean_absolute_percentage_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
bias_percent = bias_percentage(y_test, predictions)

print("MAPE with {} columns: {:.2f}%".format(n_cols, mape * 100.0))
print("MAE with {} columns: {:.2f}".format(n_cols, mae))
print("BIAS% with {} columns: {:.2f}%".format(n_cols, bias_percent * 100))

In [ ]:
# param_test1 = {
#  'max_depth':range(3,10,2),
#  'min_child_weight':range(1,6,2)
# }
# gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
#  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
#  param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch1.fit(train[predictors],train[target])
# gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

# ==================================================

In [ ]:
match_per_round = df['matchweek'].value_counts().to_frame()
last_matchweek = match_per_round[match_per_round > 5].sort_values('matchweek', ascending=False).iloc[0].name

In [ ]:
next_matches = pd.read_csv('data/all_matches.csv')

In [ ]:
next_matches['Score'] = next_matches['Score'].fillna('')

In [ ]:
next_matches = next_matches[next_matches['Score'] == '']
next_matches['Wk'] = next_matches['Wk'].fillna('')
next_matches = next_matches[next_matches['Wk'] != '']

In [ ]:
next_matches = next_matches[next_matches['Wk'] > last_matchweek]

In [ ]:
for next_match in next_matches.iterrows():
    next_match = next_match[1].to_frame().transpose()
    team1, team2 = next_match[['Home', 'Away']].iloc[0]
    date = pd.to_datetime(next_match['Date'].iloc[0])
    time = next_match['Time'].iloc[0].split(':')[0]
    matchweek = int(next_match['Wk'].iloc[0])
    
    if matchweek > last_matchweek + 1:
        break
        
    cols = X_new.columns.tolist()
    mean_features_team1 = df_teams[df_teams['team'] == team_dict[team1]][cols]
    mean_features_team2 = df_teams[df_teams['team'] == team_dict[team2]][cols]
    
    for col in cols:
        mean_features_team1[col] = mean_features_team1[col].shift(1).rolling(window=3).mean()
        mean_features_team2[col] = mean_features_team2[col].shift(1).rolling(window=3).mean()

    mean_features_team1 = mean_features_team1.iloc[-1:]
    mean_features_team2 = mean_features_team2.iloc[-1:]
    
    game_prediction = mean_features_team1.append(mean_features_team2)
    prediction = [round(prediction) for prediction in list(model.predict(game_prediction))]
    print(f"Rodada {matchweek}: {team1} {prediction[0]} x {prediction[1]} {team2}!")